In [ ]:
!sudo apt-get purge cuda
!sudo apt-get purge libcudnn6
!sudo apt-get purge libcudnn6-dev

!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.0.5.15-1+cuda9.0_amd64.deb
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7-dev_7.0.5.15-1+cuda9.0_amd64.deb
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libnccl2_2.1.4-1+cuda9.0_amd64.deb
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libnccl-dev_2.1.4-1+cuda9.0_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
!sudo apt-get update
!sudo apt-get install cuda=9.0.176-1
!sudo apt-get install libcudnn7-dev
!sudo apt-get install libnccl-dev

In [ ]:
!pip show tensorflow

In [ ]:
import cv2
import os
import tempfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from time import time
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from google.colab import files
from zipfile import ZipFile
from PIL import Image
from numpy import asarray
import numpy as np

In [ ]:
cp -r /tmp/frozen_graph/frozen_resnet502.pb /content/drive/MyDrive/frozen_graph/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/resnet50avg.h5', compile=True)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
def get_compiled_model():
    base_model = ResNet50(weights = "imagenet", include_top=False, input_shape=(256,256,3))
    base_model.trainable = True
    for layer in base_model.layers[:35]:
        layer.trainable = False
    model = Sequential([
        base_model,
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    return model

In [ ]:
model = get_compiled_model()
model.summary()

/usr/local/lib/python3.7/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 4, 4, 2048)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 2, 2, 2048)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 40965     
Total params: 23,628,677
Trainable params: 23,365,893
Non-trainable params: 262,784
_________________________________________________________________


In [ ]:
tf.keras.backend.set_learning_phase(0)

# set up tensorflow saver object
saver = tf.train.Saver()

# fetch the tensorflow session using the Keras backend
tf_session = tf.keras.backend.get_session()
# get the tensorflow session graph
input_graph_def = tf_session.graph.as_graph_def()


# write out tensorflow checkpoint & inference graph for use with freeze_graph script
saver.save(tf_session, '/tmp/frozen_graph/InceptionV3.ckpt')
tf.train.write_graph(input_graph_def, '/tmp/InceptionV3.h5', '/tmp/frozen_graph/InceptionV3.pb', as_text=False)

'/tmp/frozen_graph/InceptionV3.pb'

In [ ]:
model.input

#output_node_names="dense/Softmax"
#output_node_names.split(",")


<tf.Tensor 'resnet50_input_1:0' shape=(?, 256, 256, 3) dtype=float32>

In [ ]:
model.output

<tf.Tensor 'dense_1/Softmax:0' shape=(?, 5) dtype=float32>

# freezing Graph
use for tf 1.15.2

In [ ]:
tf.keras.backend.set_learning_phase(0)
saver = tf.train.import_meta_graph('/tmp/frozen_graph/InceptionV3.ckpt.meta', clear_devices=True)

tf_session = tf.keras.backend.get_session()
input_graph_def = tf_session.graph.as_graph_def()
sess = tf.Session()
saver.restore(sess,'/tmp/frozen_graph/InceptionV3.ckpt' )

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/frozen_graph/InceptionV3.ckpt


In [ ]:
output_node_names="dense_1/Softmax"
output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session
            input_graph_def, # input_graph_def is useful for retrieving the nodes 
            output_node_names.split(",") 
)

INFO:tensorflow:Froze 320 variables.
INFO:tensorflow:Converted 320 variables to const ops.


In [ ]:
output_graph="/tmp/frozen_graph/frozen_graph.pb"
with tf.io.gfile.GFile(output_graph, "wb") as f:
    f.write(output_graph_def.SerializeToString())
 
sess.close()

# freezing graph 2.0
use for tf 1.12

In [ ]:
model.input

<tf.Tensor 'inception_v3_input:0' shape=(?, 256, 256, 3) dtype=float32>

In [ ]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import os.path as osp

#model = load_model('/content/drive/My Drive/Colab Notebooks/Transfer Learning experiments/best model/resnet50 tf 1.12.hdf5')
#nb_classes = 5 # The number of output nodes in the model
#prefix_output_node_names_of_final_network = 'output_node'

K.set_learning_phase(0)

sess = K.get_session()
output_fld = '/tmp/frozen_graph'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
output_graph_name = '/tmp/frozen_graph/Frozen_ResNetAVG' + '.pb'
output_graph_suffix = '_inference'

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(),['dense_1/Softmax'] )
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))

INFO:tensorflow:Froze 320 variables.
INFO:tensorflow:Converted 320 variables to const ops.
saved the constant graph (ready for inference) at:  /tmp/frozen_graph/Frozen_ResNetAVG.pb


# Load model saved
see output node name  


In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile
GRAPH_PB_PATH = '/tmp/frozen_graph/frozen_graph.pb'
with tf.Session() as sess:
   print("load graph")
   with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
       graph_def = tf.GraphDef()
   graph_def.ParseFromString(f.read())
   sess.graph.as_default()
   tf.import_graph_def(graph_def, name='')
   graph_nodes=[n for n in graph_def.node]
   names = []
   for t in graph_nodes:
      names.append(t.name)
   print(names)


load graph
['conv1_1/kernel', 'conv1_1/bias', 'bn_conv1_1/gamma', 'bn_conv1_1/beta', 'bn_conv1_1/moving_mean', 'bn_conv1_1/moving_variance', 'res2a_branch2a_1/kernel', 'res2a_branch2a_1/bias', 'bn2a_branch2a_1/gamma', 'bn2a_branch2a_1/beta', 'bn2a_branch2a_1/moving_mean', 'bn2a_branch2a_1/moving_variance', 'res2a_branch2b_1/kernel', 'res2a_branch2b_1/bias', 'bn2a_branch2b_1/gamma', 'bn2a_branch2b_1/beta', 'bn2a_branch2b_1/moving_mean', 'bn2a_branch2b_1/moving_variance', 'res2a_branch2c_1/kernel', 'res2a_branch2c_1/bias', 'bn2a_branch2c_1/gamma', 'bn2a_branch2c_1/beta', 'bn2a_branch2c_1/moving_mean', 'bn2a_branch2c_1/moving_variance', 'res2a_branch1_1/kernel', 'res2a_branch1_1/bias', 'bn2a_branch1_1/gamma', 'bn2a_branch1_1/beta', 'bn2a_branch1_1/moving_mean', 'bn2a_branch1_1/moving_variance', 'res2b_branch2a_1/kernel', 'res2b_branch2a_1/bias', 'bn2b_branch2a_1/gamma', 'bn2b_branch2a_1/beta', 'bn2b_branch2a_1/moving_mean', 'bn2b_branch2a_1/moving_variance', 'res2b_branch2b_1/kernel', 're